In [ ]:
from sqlalchemy import select
import os
import json
from tca.database.models import (
    DocumentChunk,
    OpenAITextEmbedding3LargeChunkEmbedding,
    OpenAITextEmbedding3LargeThemeEmbedding,
    Theme,
)
from tca.database.session_manager import PostgresSessionManager
from sqlalchemy import func

from tca.text.llm_clients import OpenAIAPIClient

postgres_session_manager = PostgresSessionManager()
session = postgres_session_manager.session

llm_client = OpenAIAPIClient(
    model_name=os.environ["OPENAI_LLM_MODEL"],
    api_key=os.environ["OPENAI_API_KEY"],
)
# llm_client = OllamaAPIClient(
#     endpoint=Url("http://localhost:11434/api/generate"),
#     model_name="llama3.1:8b-instruct-fp16",
#     timeout=60,
# )

NB_CHUNK = 6

query = select(
    Theme,
    OpenAITextEmbedding3LargeThemeEmbedding.embeddings,
).join(Theme, OpenAITextEmbedding3LargeThemeEmbedding.theme_id == Theme.id)
theme_query_result = session.execute(query).all()

theme_list = []
doc_sentences = {}
for theme_info, query_embeddings in theme_query_result:
    theme = theme_info.themes[-1]
    theme_list.append(f"{theme} : {theme_info.description}")

    chunk_select_query = (
        select(
            DocumentChunk,
            OpenAITextEmbedding3LargeChunkEmbedding.embeddings.cosine_distance(
                query_embeddings
            ).label("cos_distance"),
            func.row_number()
            .over(
                partition_by=DocumentChunk.document_id,
                order_by=OpenAITextEmbedding3LargeChunkEmbedding.embeddings.cosine_distance(
                    query_embeddings
                ),
            )
            .label("row_number"),
        )
        .join(
            OpenAITextEmbedding3LargeChunkEmbedding,
            OpenAITextEmbedding3LargeChunkEmbedding.chunk_id == DocumentChunk.id,
        )
        .filter(
            OpenAITextEmbedding3LargeChunkEmbedding.embeddings.cosine_distance(
                query_embeddings
            )
            < 0.75
        )
        .subquery()
    )

    query = (
        select(
            chunk_select_query.c.document_id,
            func.array_agg(chunk_select_query.c.chunk_text).label("chunk_texts"),
            func.array_agg(chunk_select_query.c.cos_distance).label("cos_distances"),
        )
        .filter(chunk_select_query.c.row_number <= NB_CHUNK)
        .group_by(chunk_select_query.c.document_id)
        .order_by(func.min(chunk_select_query.c.cos_distance))
    )

    results = session.execute(query).all()

    for document_id, chunk_texts, _cos_distances in results:
        doc_sentences.setdefault(document_id, set()).update(chunk_texts)

themes_str = "\n- ".join(theme_list)
found_themes_per_doc = {}
for document_id, sentences in doc_sentences.items():
    print(f"document_id {document_id}")
    sentences = "\n- ".join(sentences)
    prompt = f"""
Je souhaite déterminerr si les thèmes suivants sont abordés explicitement dans un accord d'entreprise.
Les thèmes sont les suivants, avec la forme "theme : description du thème" :
- {themes_str}

Ces phrases proviennent de l'accord d'entreprise :

- {sentences}

Garde chaque thème qui est abordé explicitement en positif ou en négatif dans au moins une des phrases et ignore les autres thèmes.
Ajoute à ta réponse la raison de ton choix de manière succincte.
Retourne le résultat sous forme de JSON (sans balises de bloc de code) avec le format suivant :

{{
    "theme1": "raison succincte",
    "theme2": "raison succincte",
    ...
}}
    """
    response = llm_client.generate_text(prompt)
    found_themes_per_doc[document_id] = json.loads(response)
print(f"found_themes_per_doc {found_themes_per_doc}")
session.close()

In [ ]:
print(json.dumps(found_themes_per_doc["dac340e0e09f58aca61868993a8d8b1f006e6af28e070aea47c2ef270195ddd3"], indent=4, ensure_ascii=False))
result = {
    "salaires": "Amélioration des conditions salariales mentionnée.",
    "commissions": "Création d'une commission ad hoc mentionnée.",
    "fin de conflit": "Protocole de fin de conflit mentionné.",
    "indemnités": "Conditions d'accès à l'indemnité des paniers repas mentionnées.",
    "droit syndical irp & expression des salariés": "Participation des organisations syndicales CGT et STC mentionnée.",
    "dispositions conditions de travail": "Conditions de travail et plannings d'intervention mentionnés.",
    "déplacement": "Temps de déplacement considéré comme temps de travail effectif mentionné.",
}

In [ ]:
from tca.database.models import DocumentChunk

doc_id = "dac340e0e09f58aca61868993a8d8b1f006e6af28e070aea47c2ef270195ddd3"
document = session.query(DocumentChunk).filter_by(document_id=doc_id).first()
if document:
    print(document.document_name)
else:
    print(f"No document found with id {document_id}")